<a href="https://colab.research.google.com/github/NevMagi/34primates-evo-cerebellum/blob/main/Colab/Primates_Cerebellum_CrusI_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Brain Catalogue Primates Cerebellum** 

*   List item
*   List item



## A project to study the evolution of the cerebellum across primate species

Neville, Roberto, Katja & Sofie






## Intro

The cerebellum is just awesome! [...]  

Here, we will study cerebellar volume across a wide range of primate species. In a first step, we will focus on the entire cerebellum, and in a next step potentially aim at segmenting cerebellar lobes.


## Data

Structural MRI data, majority T2w scans from post mortem specimens.  
  
The image shows a sagittal slice of the brain of a hamadryas baboon, and the segmented cerebellum.

<img width="500" src="https://drive.google.com/uc?id=14_123wWGgO8DlNDKXL0fO93nCaBHL66M" /> <img width="500" src="https://drive.google.com/uc?id=1lwNRWwVN47Vy6_fDEAhHmEJ75Uu46ijx" />

## Methods

We segmented the cerebellum of 34 primate species including 65 speciemens, using structural MRI data from Heuer et al. ([2019](https://doi.org/10.1016/j.cortex.2019.04.011)). We obtained cerebellar masks using a semi-automatic approach including the open source tools SterotaxicRAMON (link) and the Web app Thresholdmann (https://neuroanatomy.github.io/thresholdmann/index.html), and then manually refined the masks using the Web app BrainBox ([https://brainbox.pasteur.fr](https://brainbox.pasteur.fr), [Heuer et al. 2016](https://doi.org/10.3897/rio.2.e9113)).

Here, we will measure the volume of the cerebellum across all specimens and investigate alterations between species.   

We programmatically obtained all cerebellum annotations using the BrainBox API. 


In [ ]:
# Install libraries
!pip install nibabel
!pip install nilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 7.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
# Import libraries
import os
from urllib.request import urlopen
import re
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.power as smp
import matplotlib.pyplot as plt
import json
import gzip
import nibabel as nib
import nilearn
from nilearn import plotting
import seaborn as sbn
import csv

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#### Get token
As the project is currently a private project with access restricted to collaborators, we need to get a token to access the project and download the annotations. Go to BrainBox at [https://brainbox.pasteur.fr/token](https://brainbox.pasteur.fr/token) to obtain your token.

In [ ]:
# Get your token (valid for 24h)
token = "bwndu8u1yj7fz1dbyvb4qj"

### Set details of the BrainBox project

In [ ]:
# Access BrainBox API to get data from the Brain Catalogue Primates Cerebellum Project
project_name = "BrainCataloguePrimatesCB"
prj_url = "https://brainbox.pasteur.fr/project/json/%s?token=%s"%(project_name, token) 
res = urlopen(prj_url)
txt = res.read()
prj = json.loads(txt)
nfiles = len(prj['files']['list'])
print(nfiles,"files")

67 files


In [ ]:
# Get individual mri annotations
masknames = ["Cerebellum_lab"]

In [ ]:
RAWSEG_DIR = "./data/"
!mkdir -p $RAWSEG_DIR

In [ ]:
# Define function for downloading segmentations from BrainBox
def loadAtlas(atlas_url):
    readable = False
    sum = 0
    img = None
    try:
        # load from url
        f = urlopen( atlas_url )
        local_file = open('tmp.nii.gz', "wb" )
        local_file.write( f.read() )
        local_file.close()

        # fix magic
        f = gzip.open('tmp.nii.gz', 'r')
        data = bytearray(f.read())
        f.close()
        data[347]=0
        local_file = open('tmp.nii', "wb" )
        local_file.write(data)
        local_file.close()

        # load file
        img = nib.load('tmp.nii')
        dim = img.shape;
        data = np.reshape(img.get_fdata(),dim[0]*dim[1]*dim[2]);
        sum = np.sum(data>0)
        readable = True
    except:
        pass
    return (img, sum, readable)

### Download data, compute volume

In [ ]:
# Download the segmentations from BrainBox
N = nfiles
data = []
for isub in range( N ):
    bb_url = "https://brainbox.pasteur.fr/mri/json?url="+prj['files']['list'][isub]+"&token="+token
    res = urlopen( bb_url )
    txt = res.read()
    mri = json.loads( txt )
    specimenID = mri['mri']['annotations'][project_name]['SpecimenID']['data']
    vox_vol = mri['pixdim'][0] * mri['pixdim'][1] * mri['pixdim'][2]
    # print( str(isub + 1) + ": " + specimenID )

    # Search for the masknames atlases within the annotations
    natlases = len(mri['mri']['atlas'])
    base = "https://brainbox.pasteur.fr"+mri['url']
    index = -1
    for i in range( 0, natlases ):
        for j in masknames:
            if (
                'name' in mri['mri']['atlas'][i] and
                mri['mri']['atlas'][i]['name'] == j and
                mri['mri']['atlas'][i]['project'] == project_name
            ):
                index = i
                filename = mri['mri']['atlas'][i]['filename']
                dst = '{dir}/{sub}'.format( dir=RAWSEG_DIR, sub=specimenID)
                if not os.path.exists( dst ):
                    os.makedirs( dst )

                # Load atlas
                atlas_url = base + filename
                # print( atlas_url )
                try:
                    f = urlopen( atlas_url )
                    local_file = open( dst + "/ROI-" + j + '_mask.nii.gz', "wb" )
                    local_file.write( f.read() )
                    local_file.close()
                    f.close()

                    _, nvox, readable = loadAtlas(atlas_url)
                    if readable:
                      CrusVol = nvox * vox_vol
                      print("{i}. '{specimenID}' {CrusVol} mm3".format(i=isub, specimenID=specimenID, CrusVol=CrusVol))
                      data.append([isub, specimenID, float(CrysVol)])
                except:
                    print(">> File does not exist: " + atlas_url)

                break
    if index<0:
        print("No atlases contain the requested masks")

0. 'ApelleCebusApell_f4b9' 1012.3342287759373 mm3
>> File does not exist: https://brainbox.pasteur.fr/data/4cd94e28daa0c1627ec2fa628fd4cf79/i19vcggowwd.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/0c611a8f2783f0c8d19ae2a71c230ce6/u8ihshx6b4q.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/fe06a1f562fd9b695ad36348dc417a6d/wxr6bkljsoq.nii.gz
3. 'AyeAyeDaubentoni_cb56' 373.97317259471447 mm3
>> File does not exist: https://brainbox.pasteur.fr/data/62d8869ba98ec331fabb851605cdad12/uchlih2ae3p.nii.gz
4. 'ColobeColobusPol_6118' 842.087440088672 mm3
>> File does not exist: https://brainbox.pasteur.fr/data/56f81183d798bf350dfabf562b40dbcf/yf7m8e577o.nii.gz
5. 'DouroucouliAotus_091c' 216.5 mm3
>> File does not exist: https://brainbox.pasteur.fr/data/36b58396b1c4ade7b3f6e6652a8719a1/sm0yhm8nhb.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/a6dddbccdc639003e49fab78587c0d47/2rp6gk7w244.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/da

# **Download data, plot some results**

In [ ]:
N = nfiles-1 # N=nfiles for doing them all, now we exclude the 1 mm isotropic human brain

In [ ]:
# Download the segmentations from BrainBox
data = []
for isub in range(N):
    bb_url = "https://brainbox.pasteur.fr/mri/json?url="+prj['files']['list'][isub]+"&token="+token
    res = urlopen(bb_url)
    txt = res.read()
    mri = json.loads(txt)
    specimenID = mri['mri']['annotations'][project_name]['SpecimenID']['data']
    vox_vol = mri['pixdim'][0] * mri['pixdim'][1] * mri['pixdim'][2]

    # Search for specific atlases within the annotations
    natlases = len(mri['mri']['atlas'])
    base = "https://brainbox.pasteur.fr"+mri['url']
    name = mri['name']

    # Search for the new Cerebellum annotation
    # http://brainbox.pasteur.fr/mri/json?url=https://drive.google.com/uc?id=1UzoxTY9Vt9djWliFeo6ib5OWdq6WE8O3
    found_filename = -1
    for i in range(0,natlases):
        if 'name' in mri['mri']['atlas'][i] and mri['mri']['atlas'][i]['name'] == 'Cerebellum_lab' and mri['mri']['atlas'][i]['project'] == project_name:
            found_filename = i
            filename = mri['mri']['atlas'][i]['filename']
    
    # Search for SpecimenID
    ntextannotations = len(mri['mri']['annotations'])
    found_id=-1
    for i in range(0,ntextannotations):
        if('annotations' in mri['mri']):
            found_id=i
            specimenID = mri['mri']['annotations'][project_name]['SpecimenID']['data']
            break
    if found_filename<0:
        print("didn't find filename")
        continue
    
    cb_img, nvox, readable = loadAtlas(base + filename)
    CrusVol = nvox * vox_vol
    print("{i}. '{specimenID}' {CrusVol} mm3".format(i=isub,specimenID=specimenID,CrusVol=CrusVol))
    data.append([isub,specimenID,float(CrusVol)])

data = np.array(data)

0. 'ApelleCebusApell_f4b9' 1012.3342287759373 mm3
1. 'Atele_6717' 0.0 mm3
2. 'AteleAtelesAter_7245' 0.0 mm3
3. 'AyeAyeDaubentoni_cb56' 373.97317259471447 mm3
4. 'ColobeColobusPol_6118' 842.087440088672 mm3
5. 'DouroucouliAotus_091c' 216.5 mm3
6. 'GalagoDemidoviiG_6cf3' 0.0 mm3
7. 'HurleurRouxAloua_47da' 0.0 mm3
8. 'LangurIndienPith_8390' 0.0 mm3
9. 'LemurMongosLemur_b941' 0.0 mm3
10. 'LepilemurAQueueR_309f' 0.0 mm3
11. 'MakiCattaLemurCa_dec8' 0.0 mm3
12. 'MangabeyCouronne_6b65' 178.81598134803804 mm3
13. 'MarsousetOedipom_e834' 0.0 mm3
14. 'MicrocebeDeCoque_e3a9' 0.0 mm3
15. 'MicrocebeMignonM_ca6c' 0.0 mm3
16. 'MoustacLasiopygi_23cc' 0.0 mm3
17. 'NycticebusTardig_08cb' 0.0 mm3
18. 'OuistitiAPinceau_b4aa' 0.0 mm3
19. 'PithecusGermaini_6a2c' 0.0 mm3
20. 'SaimiriCassiquia_7176' 103.125 mm3
21. 'SaimiriCassiquia_eb44' 0.0 mm3
22. 'SingeLaineuxLago_0f84' 873.75 mm3
23. 'VariNoirEtBlancL_4d0e' 0.0 mm3
24. 'VervetCercopithe_72ae' 505.41306024992707 mm3
25. 'BabouinCercopith_a723' 1841.4459562

## Display the last brain and its cerebellum

### Plot cerebellar volume

In [ ]:
# Create data frame from the downloaded data
dfCrusNeville = pd.DataFrame(data={'id':data[:,0],'SpecimenID':data[:,1],'CrusVol':data[:,2].astype(float)})


In [ ]:
pd.set_option('display.max_rows', None)
dfCrusNeville

,id,SpecimenID,CrusVol
0,0,ApelleCebusApell_f4b9,1012.334229
1,1,Atele_6717,0.000000
2,2,AteleAtelesAter_7245,0.000000
3,3,AyeAyeDaubentoni_cb56,373.973173
4,4,ColobeColobusPol_6118,842.087440
5,5,DouroucouliAotus_091c,216.500000
6,6,GalagoDemidoviiG_6cf3,0.000000
7,7,HurleurRouxAloua_47da,0.000000
8,8,LangurIndienPith_8390,0.000000
9,9,LemurMongosLemur_b941,0.000000


In [ ]:
# sbn.distplot(df["cbvol"])
dfcpCrusNeville <- df.loc[dfCrusNeville['CrusVol'] >= 1]

# sbn.displot(df["cbvol"])
sbn.histplot(dfcpCrusNeville["CrusVol"])

#plotting.plot_roi(cb_img2, bg_img=mri_img)
# plotting.plot_anat(cb_img2)

NameError: ignored

In [ ]:
#?df.to_csv

#print(df.dtypes)
#df2 = df
#df2["cbvol"] = df["cbvol"].astype(object)
#print(df2)
#print(df2.dtypes)

from google.colab import files

dfCrusNeville.to_csv('dfCrusNeville.csv', sep=',', decimal='.')
files.download('dfCrusNeville.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Now download Mahta's Crus I/II volumes**


In [ ]:
# Get individual mri annotations
masknames = ["lobule_mahta"]

In [ ]:
# Download the segmentations from BrainBox
N = nfiles
data = []
for isub in range( N ):
    bb_url = "https://brainbox.pasteur.fr/mri/json?url="+prj['files']['list'][isub]+"&token="+token
    res = urlopen( bb_url )
    txt = res.read()
    mri = json.loads( txt )
    specimenID = mri['mri']['annotations'][project_name]['SpecimenID']['data']
    vox_vol = mri['pixdim'][0] * mri['pixdim'][1] * mri['pixdim'][2]
    # print( str(isub + 1) + ": " + specimenID )

    # Search for the masknames atlases within the annotations
    natlases = len(mri['mri']['atlas'])
    base = "https://brainbox.pasteur.fr"+mri['url']
    index = -1
    for i in range( 0, natlases ):
        for j in masknames:
            if (
                'name' in mri['mri']['atlas'][i] and
                mri['mri']['atlas'][i]['name'] == j and
                mri['mri']['atlas'][i]['project'] == project_name
            ):
                index = i
                filename = mri['mri']['atlas'][i]['filename']
                dst = '{dir}/{sub}'.format( dir=RAWSEG_DIR, sub=specimenID)
                if not os.path.exists( dst ):
                    os.makedirs( dst )

                # Load atlas
                atlas_url = base + filename
                # print( atlas_url )
                try:
                    f = urlopen( atlas_url )
                    local_file = open( dst + "/ROI-" + j + '_mask.nii.gz', "wb" )
                    local_file.write( f.read() )
                    local_file.close()
                    f.close()

                    _, nvox, readable = loadAtlas(atlas_url)
                    if readable:
                      CrusVol = nvox * vox_vol
                      print("{i}. '{specimenID}' {CrusVol} mm3".format(i=isub, specimenID=specimenID, CrusVol=CrusVol))
                      data.append([isub, specimenID, float(CrysVol)])
                except:
                    print(">> File does not exist: " + atlas_url)

                break
    if index<0:
        print("No atlases contain the requested masks")

>> File does not exist: https://brainbox.pasteur.fr/data/4cd94e28daa0c1627ec2fa628fd4cf79/esy2vvobzfo.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/0c611a8f2783f0c8d19ae2a71c230ce6/ui3xtyir1km.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/fe06a1f562fd9b695ad36348dc417a6d/q6nfa3kobng.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/62d8869ba98ec331fabb851605cdad12/grfj4c92dve.nii.gz
4. 'ColobeColobusPol_6118' 836.09491480339 mm3
>> File does not exist: https://brainbox.pasteur.fr/data/56f81183d798bf350dfabf562b40dbcf/3ghusoac8g4.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/36b58396b1c4ade7b3f6e6652a8719a1/eqqave0t5lf.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/a6dddbccdc639003e49fab78587c0d47/gkydnjzigzs.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/b5afdf839b52ea985356abd887c70dd3/0p9gimdbnxym.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/20b6b6c1f1bc142791b234a

In [ ]:
N = nfiles-1 # N=nfiles for doing them all

In [ ]:
# Download the segmentations from BrainBox
data = []
for isub in range(N):
    bb_url = "https://brainbox.pasteur.fr/mri/json?url="+prj['files']['list'][isub]+"&token="+token
    res = urlopen(bb_url)
    txt = res.read()
    mri = json.loads(txt)
    specimenID = mri['mri']['annotations'][project_name]['SpecimenID']['data']
    vox_vol = mri['pixdim'][0] * mri['pixdim'][1] * mri['pixdim'][2]

    # Search for specific atlases within the annotations
    natlases = len(mri['mri']['atlas'])
    base = "https://brainbox.pasteur.fr"+mri['url']
    name = mri['name']

    # Search for the new Cerebellum annotation
    # http://brainbox.pasteur.fr/mri/json?url=https://drive.google.com/uc?id=1UzoxTY9Vt9djWliFeo6ib5OWdq6WE8O3
    found_filename = -1
    for i in range(0,natlases):
        if 'name' in mri['mri']['atlas'][i] and mri['mri']['atlas'][i]['name'] == 'lobule_mahta' and mri['mri']['atlas'][i]['project'] == project_name:
            found_filename = i
            filename = mri['mri']['atlas'][i]['filename']
    
    # Search for SpecimenID
    ntextannotations = len(mri['mri']['annotations'])
    found_id=-1
    for i in range(0,ntextannotations):
        if('annotations' in mri['mri']):
            found_id=i
            specimenID = mri['mri']['annotations'][project_name]['SpecimenID']['data']
            break
    if found_filename<0:
        print("didn't find filename")
        continue
    
    cb_img, nvox, readable = loadAtlas(base + filename)
    CrusVol = nvox * vox_vol
    print("{i}. '{specimenID}' {CrusVol} mm3".format(i=isub,specimenID=specimenID,CrusVol=CrusVol))
    data.append([isub,specimenID,float(CrusVol)])

data = np.array(data)

0. 'ApelleCebusApell_f4b9' 0.0 mm3
1. 'Atele_6717' 0.0 mm3
2. 'AteleAtelesAter_7245' 0.0 mm3
3. 'AyeAyeDaubentoni_cb56' 0.0 mm3
4. 'ColobeColobusPol_6118' 836.09491480339 mm3
5. 'DouroucouliAotus_091c' 0.0 mm3
6. 'GalagoDemidoviiG_6cf3' 0.0 mm3
7. 'HurleurRouxAloua_47da' 0.0 mm3
8. 'LangurIndienPith_8390' 0.0 mm3
9. 'LemurMongosLemur_b941' 0.0 mm3
10. 'LepilemurAQueueR_309f' 0.0 mm3
11. 'MakiCattaLemurCa_dec8' 0.0 mm3
12. 'MangabeyCouronne_6b65' 0.0 mm3
13. 'MarsousetOedipom_e834' 0.0 mm3
14. 'MicrocebeDeCoque_e3a9' 0.0 mm3
15. 'MicrocebeMignonM_ca6c' 0.0 mm3
16. 'MoustacLasiopygi_23cc' 0.0 mm3
didn't find filename
18. 'OuistitiAPinceau_b4aa' 0.0 mm3
19. 'PithecusGermaini_6a2c' 0.0 mm3
20. 'SaimiriCassiquia_7176' 0.0 mm3
21. 'SaimiriCassiquia_eb44' 0.0 mm3
22. 'SingeLaineuxLago_0f84' 782.796875 mm3
23. 'VariNoirEtBlancL_4d0e' 0.0 mm3
24. 'VervetCercopithe_72ae' 538.1640641541507 mm3
25. 'BabouinCercopith_a723' 0.0 mm3
26. 'MacacaFascicular_3a21' 0.0 mm3
27. 'GorillaBeringeiG_0854' 0.0 

In [ ]:
# Create data frame from the downloaded data
dfCrusMahta = pd.DataFrame(data={'id':data[:,0],'SpecimenID':data[:,1],'CrusVol':data[:,2].astype(float)})

In [ ]:
pd.set_option('display.max_rows', None)
dfCrusMahta

,id,SpecimenID,CrusVol
0,0,ApelleCebusApell_f4b9,0.000000
1,1,Atele_6717,0.000000
2,2,AteleAtelesAter_7245,0.000000
3,3,AyeAyeDaubentoni_cb56,0.000000
4,4,ColobeColobusPol_6118,836.094915
5,5,DouroucouliAotus_091c,0.000000
6,6,GalagoDemidoviiG_6cf3,0.000000
7,7,HurleurRouxAloua_47da,0.000000
8,8,LangurIndienPith_8390,0.000000
9,9,LemurMongosLemur_b941,0.000000


In [ ]:
dfCrusMahta.to_csv('dfCrusMahta.csv', sep=',', decimal='.')
files.download('dfCrusMahta.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Now download Vanessa's Crus I/II volumes**

In [ ]:
# Get individual mri annotations
masknames = ["lobule_nele"]

In [ ]:
# Download the segmentations from BrainBox
N = nfiles
data = []
for isub in range( N ):
    bb_url = "https://brainbox.pasteur.fr/mri/json?url="+prj['files']['list'][isub]+"&token="+token
    res = urlopen( bb_url )
    txt = res.read()
    mri = json.loads( txt )
    specimenID = mri['mri']['annotations'][project_name]['SpecimenID']['data']
    vox_vol = mri['pixdim'][0] * mri['pixdim'][1] * mri['pixdim'][2]
    # print( str(isub + 1) + ": " + specimenID )

    # Search for the masknames atlases within the annotations
    natlases = len(mri['mri']['atlas'])
    base = "https://brainbox.pasteur.fr"+mri['url']
    index = -1
    for i in range( 0, natlases ):
        for j in masknames:
            if (
                'name' in mri['mri']['atlas'][i] and
                mri['mri']['atlas'][i]['name'] == j and
                mri['mri']['atlas'][i]['project'] == project_name
            ):
                index = i
                filename = mri['mri']['atlas'][i]['filename']
                dst = '{dir}/{sub}'.format( dir=RAWSEG_DIR, sub=specimenID)
                if not os.path.exists( dst ):
                    os.makedirs( dst )

                # Load atlas
                atlas_url = base + filename
                # print( atlas_url )
                try:
                    f = urlopen( atlas_url )
                    local_file = open( dst + "/ROI-" + j + '_mask.nii.gz', "wb" )
                    local_file.write( f.read() )
                    local_file.close()
                    f.close()

                    _, nvox, readable = loadAtlas(atlas_url)
                    if readable:
                      CrusVol = nvox * vox_vol
                      print("{i}. '{specimenID}' {CrusVol} mm3".format(i=isub, specimenID=specimenID, CrusVol=CrusVol))
                      data.append([isub, specimenID, float(CrysVol)])
                except:
                    print(">> File does not exist: " + atlas_url)

                break
    if index<0:
        print("No atlases contain the requested masks")

>> File does not exist: https://brainbox.pasteur.fr/data/4cd94e28daa0c1627ec2fa628fd4cf79/l5dlf61bbu.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/0c611a8f2783f0c8d19ae2a71c230ce6/ed1rv5ag8en.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/fe06a1f562fd9b695ad36348dc417a6d/3dhkwu7xipp.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/62d8869ba98ec331fabb851605cdad12/cyni0owdi2.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/56f81183d798bf350dfabf562b40dbcf/cjwaemv4dzt.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/36b58396b1c4ade7b3f6e6652a8719a1/rzxy3v6w3gj.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/a6dddbccdc639003e49fab78587c0d47/y361hq5c0e.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/b5afdf839b52ea985356abd887c70dd3/kb7wyi00kp.nii.gz
>> File does not exist: https://brainbox.pasteur.fr/data/20b6b6c1f1bc142791b234a692cfc4a4/x4dddh2wcec.nii.gz
>> File does not exist:

In [ ]:
N = nfiles-1 # N=nfiles for doing them all

In [ ]:
# Download the segmentations from BrainBox
data = []
for isub in range(N):
    bb_url = "https://brainbox.pasteur.fr/mri/json?url="+prj['files']['list'][isub]+"&token="+token
    res = urlopen(bb_url)
    txt = res.read()
    mri = json.loads(txt)
    specimenID = mri['mri']['annotations'][project_name]['SpecimenID']['data']
    vox_vol = mri['pixdim'][0] * mri['pixdim'][1] * mri['pixdim'][2]

    # Search for specific atlases within the annotations
    natlases = len(mri['mri']['atlas'])
    base = "https://brainbox.pasteur.fr"+mri['url']
    name = mri['name']

    # Search for the new Cerebellum annotation
    # http://brainbox.pasteur.fr/mri/json?url=https://drive.google.com/uc?id=1UzoxTY9Vt9djWliFeo6ib5OWdq6WE8O3
    found_filename = -1
    for i in range(0,natlases):
        if 'name' in mri['mri']['atlas'][i] and mri['mri']['atlas'][i]['name'] == 'lobule_nele' and mri['mri']['atlas'][i]['project'] == project_name:
            found_filename = i
            filename = mri['mri']['atlas'][i]['filename']
    
    # Search for SpecimenID
    ntextannotations = len(mri['mri']['annotations'])
    found_id=-1
    for i in range(0,ntextannotations):
        if('annotations' in mri['mri']):
            found_id=i
            specimenID = mri['mri']['annotations'][project_name]['SpecimenID']['data']
            break
    if found_filename<0:
        print("didn't find filename")
        continue
    
    cb_img, nvox, readable = loadAtlas(base + filename)
    CrusVol = nvox * vox_vol
    print("{i}. '{specimenID}' {CrusVol} mm3".format(i=isub,specimenID=specimenID,CrusVol=CrusVol))
    data.append([isub,specimenID,float(CrusVol)])

data = np.array(data)

0. 'ApelleCebusApell_f4b9' 0.0 mm3
1. 'Atele_6717' 0.0 mm3
2. 'AteleAtelesAter_7245' 0.0 mm3
3. 'AyeAyeDaubentoni_cb56' 0.0 mm3
4. 'ColobeColobusPol_6118' 0.0 mm3
5. 'DouroucouliAotus_091c' 0.0 mm3
6. 'GalagoDemidoviiG_6cf3' 0.0 mm3
7. 'HurleurRouxAloua_47da' 0.0 mm3
8. 'LangurIndienPith_8390' 0.0 mm3
9. 'LemurMongosLemur_b941' 0.0 mm3
10. 'LepilemurAQueueR_309f' 0.0 mm3
11. 'MakiCattaLemurCa_dec8' 0.0 mm3
12. 'MangabeyCouronne_6b65' 0.0 mm3
13. 'MarsousetOedipom_e834' 0.0 mm3
14. 'MicrocebeDeCoque_e3a9' 0.0 mm3
15. 'MicrocebeMignonM_ca6c' 0.0 mm3
16. 'MoustacLasiopygi_23cc' 0.0 mm3
didn't find filename
18. 'OuistitiAPinceau_b4aa' 0.0 mm3
19. 'PithecusGermaini_6a2c' 0.0 mm3
20. 'SaimiriCassiquia_7176' 0.0 mm3
21. 'SaimiriCassiquia_eb44' 0.0 mm3
22. 'SingeLaineuxLago_0f84' 0.0 mm3
23. 'VariNoirEtBlancL_4d0e' 0.0 mm3
24. 'VervetCercopithe_72ae' 0.0 mm3
25. 'BabouinCercopith_a723' 0.0 mm3
26. 'MacacaFascicular_3a21' 0.0 mm3
27. 'GorillaBeringeiG_0854' 0.0 mm3
28. 'MangabeyCercoceb_4458' 0

In [ ]:
# Create data frame from the downloaded data
dfCrusVanessa = pd.DataFrame(data={'id':data[:,0],'SpecimenID':data[:,1],'CrusVol':data[:,2].astype(float)})

In [ ]:
pd.set_option('display.max_rows', None)
dfCrusVanessa

,id,SpecimenID,CrusVol
0,0,ApelleCebusApell_f4b9,0.0
1,1,Atele_6717,0.0
2,2,AteleAtelesAter_7245,0.0
3,3,AyeAyeDaubentoni_cb56,0.0
4,4,ColobeColobusPol_6118,0.0
5,5,DouroucouliAotus_091c,0.0
6,6,GalagoDemidoviiG_6cf3,0.0
7,7,HurleurRouxAloua_47da,0.0
8,8,LangurIndienPith_8390,0.0
9,9,LemurMongosLemur_b941,0.0


In [ ]:
dfCrusVanessa.to_csv('dfCrusVanessa.csv', sep=',', decimal='.')
files.download('dfCrusVanessa.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>